In [5]:
# Importar librerias
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.api as sm # pip install statsmodels
from datetime import timedelta

In [6]:
# Parametros de descarga
ticker = "TSLA"
benchmark_ticker = "^GSPC" # Ticker de indice de referencia (S&P500)
fecha_inicial = "2020-01-01"
fecha_final = "2024-01-01"

In [7]:
# Descargar Datos
datos = yf.download(ticker, start=fecha_inicial, end=fecha_final, interval="1d", multi_level_index=False)
benchmark = yf.download(benchmark_ticker, start=fecha_inicial, end=fecha_final, interval="1d", multi_level_index=False)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [8]:
# Calcular los rendimientos diarios
datos["Rendimiento"] = datos["Close"].pct_change()
benchmark["Rendimiento"] = benchmark["Close"].pct_change()
datos.dropna()
benchmark.dropna()

,Close,High,Low,Open,Volume,Rendimiento
Date,,,,,,
2020-01-03,3234.850098,3246.149902,3222.340088,3226.360107,3484700000,-0.007060
2020-01-06,3246.280029,3246.840088,3214.639893,3217.550049,3702460000,0.003533
2020-01-07,3237.179932,3244.909912,3232.429932,3241.860107,3435910000,-0.002803
2020-01-08,3253.050049,3267.070068,3236.669922,3238.590088,3726840000,0.004902
2020-01-09,3274.699951,3275.580078,3263.669922,3266.030029,3641230000,0.006655
...,...,...,...,...,...,...
2023-12-22,4754.629883,4772.939941,4736.770020,4753.919922,3046770000,0.001660
2023-12-26,4774.750000,4784.720215,4758.450195,4758.859863,2513910000,0.004232
2023-12-27,4781.580078,4785.390137,4768.899902,4773.450195,2748450000,0.001430


In [9]:
# Calcular MDD Max DRAWDOWN. 
# La maxima perdida es la caída máxima que experimenta una inversión durante un periodo determinado. 
# Es un indicador financiero que se utiliza para medir el riesgo de un activo. 
def max_drawdown(rendimientos):
    
    """
    Calcula la máxima pérdida de una serie de rendimientos
    """
    
    # Calcular
    capital_acumulado = (1 + rendimientos).cumprod()
    maximo_acumulado = capital_acumulado.cummax()
    drawdown = (maximo_acumulado - capital_acumulado) / maximo_acumulado
    maxima_perdida_valor = drawdown.max()
    
    # Encontrar el punto más alto y el punto más bajo del drawdown
    fecha_final = drawdown.idxmax()
    fecha_maxima = capital_acumulado[:fecha_final].idxmax()
    
    return maxima_perdida_valor, capital_acumulado, drawdown, fecha_maxima, fecha_final

In [12]:
maxima_perdida_accion, capital_acumulado, drawdown, fecha_maxima, fecha_final = max_drawdown(datos["Rendimiento"])


In [13]:
# print(f"Maxima Perdida de {ticker}: {maxima_perdida_accion:.2%}")
print(f"Máxima Pérdida de {ticker}: {maxima_perdida_accion:.2%}")

Máxima Pérdida de TSLA: 73.63%


In [14]:
# Comentario explicativo
print(f"""
La Máxima Pérdida es la mayor pérdida desde un punto alto hasta un punto bajo durante el periodo analizado.
En este caso, la Máxima Pérdida del ticker: {ticker} es {maxima_perdida_accion:.2%}.
Esto indica la mayor caída en el valor de la acción desde un máxima histórico durante el periodo de estudio.
      """)


La Máxima Pérdida es la mayor pérdida desde un punto alto hasta un punto bajo durante el periodo analizado.
En este caso, la Máxima Pérdida del ticker: TSLA es 73.63%.
Esto indica la mayor caída en el valor de la acción desde un máxima histórico durante el periodo de estudio.
      


In [ ]:
# Graficar la maxima perdida
plt.figure(figsize=(22,8))
plt.plot(datos.index, capital_acumulado, label="Rendimiento Acumulado", color="blue")
plt.plot(datos.index, capital_acumulado - drawdown *capital_acumulado, label="Drawdown", color="red", linestyle="--")

# Resaltar el área de Máxima Pérdida desde el punto más alto hasta el más bajo

